In [96]:
import pandas as pd
import yaml

# Load config file
with open("config.yaml", "r") as ymlfile:
    cfg = yaml.safe_load(ymlfile)


market_cap = pd.read_excel(cfg["Tickers_path"], sheet_name="MarketCaps")

# Dictionnaire de correspondance
correspondance = pd.read_excel(cfg["Tickers_path"], sheet_name="Mapping")
sedols_to_tickers = correspondance[["Sedol", "Tickers"]]
sedol_to_ticker = dict(zip(sedols_to_tickers["Sedol"], sedols_to_tickers["Tickers"]))

prices = pd.read_csv(cfg["Prices_path"], index_col=0)

market_cap = market_cap.set_index("Date")
market_cap.columns = [sedol_to_ticker[col] for col in market_cap.columns]

market_cap = market_cap.resample('D').ffill()
cutoff_day_start = prices.index[0]
cutoff_day_end = prices.index[-1]

market_cap = market_cap[market_cap.index >= cutoff_day_start]
market_cap = market_cap[market_cap.index <= cutoff_day_end]

prices.index = pd.to_datetime(prices.index)    
market_cap.index = pd.to_datetime(market_cap.index)

In [106]:
df_merged = pd.merge(market_cap, prices, on='Date', how='inner')


print(df_merged["AMZN_x"] * df_merged["AMZN_y"])

Date
2010-01-04    0.035406
2010-01-05    0.035615
2010-01-06    0.034970
2010-01-07    0.034375
2010-01-08    0.035306
                ...   
2019-12-24    2.889140
2019-12-26    3.017610
2019-12-27    3.019273
2019-12-30    2.982279
2019-12-31    2.993225
Length: 2516, dtype: float64
